***JOH ACKNOWLEDGE AUDIT DATA***

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

keyvault_name = f"ingest{lz_key}-meta002-{env}"

# Access the Service Principle secrets from keyvaults
client_secret = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-SECRET')
tenant_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-TENANT-ID')
client_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-ID')

storage_accounts = ["curated"]

In [0]:
for storage_account in storage_accounts:
    spark.conf.set(
        f'fs.azure.account.auth.type.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', 'OAuth')
    spark.conf.set(
        f'fs.azure.account.oauth.provider.type.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
    spark.conf.set(
        f'fs.azure.account.oauth2.client.id.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', client_id)
    spark.conf.set(
        f'fs.azure.account.oauth2.client.secret.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', client_secret)
    spark.conf.set(
        f'fs.azure.account.oauth2.client.endpoint.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', f'https://login.microsoftonline.com/{tenant_id}/oauth2/token')

In [0]:
curated_storage_account = f"ingest{lz_key}curated{env}"
silver_container = "silver" 

In [0]:
df_joh_ack_audit_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/AUDIT/JOH/joh_ack_audit/")
df_joh_ack_audit_data.createOrReplaceTempView("joh_acknowledge_audit_data")

In [0]:
%sql 
SELECT DATE_FORMAT(timestamp, 'ddMMMyyyy') as date,
       COUNT(filename) as total_expected_count,
       COUNT(CASE WHEN http_response = 201  and http_message = 'Created' THEN 1 END) as count_successful_response,
       COUNT(CASE WHEN http_response <> 201 and http_message <> 'Created' THEN 1 END) as count_unsuccessful_response
       
FROM joh_acknowledge_audit_data
GROUP BY DATE_FORMAT(timestamp, 'ddMMMyyyy')

Databricks visualization. Run in Databricks to view.

***JOH CREATE AUDIT DATA***

In [0]:
df_joh_created_audit_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/AUDIT/JOH/joh_cr_audit_table/")

df_joh_created_audit_data.createOrReplaceTempView("joh_created_audit_data")

In [0]:
%sql

SELECT * FROM joh_created_audit_data



***Eventhub AUDIT DATA***

In [0]:
df_publish_audit_db_eh_audit_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/AUDIT/JOH/JOH_HTML_a360_pub_audit_db_eh/")
df_publish_audit_db_eh_audit_data.createOrReplaceTempView("joh_publish_audit_db_eh_data")

In [0]:
%sql 

SELECT 
        COUNT(CASE WHEN status = 'success' THEN 1 END) AS count_successful_response,
        COUNT(CASE WHEN status <> 'success' THEN 1 END) AS count_unsuccessful_response

FROM joh_publish_audit_db_eh_data

Databricks visualization. Run in Databricks to view.